In [ ]:
!pip install unsloth # install unsloth
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git # Also get the latest version Unsloth!

print("Done")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.7/192.7 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-req-build-wcnao2jp
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-req-build-wcnao2jp
  Resolved https://github.com/unslothai/unsloth.git to commit 6c234d5a66adb76b9b93fb0f2445648199d88e66
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2025.3.19-py3-none-any.whl size=192661 sha256=34b87b5d3af3830173d904e86e0039664234325156323a87660773582279b118
  Stored in directory: /tmp/pip-ephem-wheel-cache-tplad22g/wheels/d1/17/05/850ab10c33284a4763b0595cd8ea9d01fce6e221cac24b3c01
Successfully built unsloth
  Attempting uninstall: unsloth
    Found existing installation: unsloth 2025.3.19
    Uninstalling unsloth-2025.3.19:
      Successfully uninstalled unsloth-2025.3.19
Done


In [ ]:
from unsloth import FastLanguageModel
import torch # Import PyTorch
from trl import SFTTrainer # Trainer for supervised fine-tuning (SFT)
from unsloth import is_bfloat16_supported # Checks if the hardware supports bfloat16 precision
# Hugging Face modules
from huggingface_hub import login # Lets you login to API
from transformers import TrainingArguments # Defines training hyperparameters
from datasets import load_dataset # Lets you load fine-tuning datasets
# Import weights and biases
import wandb


print("Done")

Done


In [ ]:
from google.colab import userdata

In [ ]:
# Initialize Hugging Face & WnB tokens
hugging_face_token = userdata.get("Hugging_Face_Token")
wnb_token = userdata.get("wnb")

# Login to Hugging Face
login(hugging_face_token) # from huggingface_hub import login

# Login to WnB
wandb.login(key=wnb_token) # import wandb
run = wandb.init(
    project='Fine-tune-DeepSeek-R1-Distill-Llama-8B on English Quotes',
    job_type="training",
    anonymous="allow"
)

print("Done")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: avhisheknandi03 (avhisheknandi03-institute-of-engineering-management-salt) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Done


In [ ]:
# Set parameters
max_seq_length = 2048 # Define the maximum sequence length a model can handle (i.e. how many tokens can be processed at once)
dtype = None # Set to default
load_in_4bit = True # Enables 4 bit quantization — a memory saving optimization

# Load the DeepSeek R1 model and tokenizer using unsloth — imported using: from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Llama-8B",  # Load the pre-trained DeepSeek R1 model (8B parameter version)
    max_seq_length=max_seq_length, # Ensure the model can process up to 2048 tokens at once
    dtype=dtype, # Use the default data type (e.g., FP16 or BF16 depending on hardware support)
    load_in_4bit=load_in_4bit, # Load the model in 4-bit quantization to save memory
    token=hugging_face_token, # Use hugging face token
)

print("Done")

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Done


In [ ]:
# Define a system prompt under prompt_style
# prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
# Write a response that appropriately completes the request.
# Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

# ### Instruction:
# You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.
# Please answer the following medical question.

# ### Question:
# {}

# ### Response:
# <think>{}"""
prompt_style = """Below is an instruction that describes a user’s emotional or motivational need, paired with a topic as input.
Write a meaningful and relevant quote that matches the given topic or emotion.
Before answering, think briefly about the mood or theme to ensure the quote is impactful and appropriate.

### Instruction:
You are a motivational quote generator trained on quotes from well-known personalities.
Given a topic, respond with an inspirational or thoughtful quote that fits well.

### Topic:
{}

### Response:
<think>{}"""

In [ ]:
# Creating a test  question for inference
question = """Gratitude in Difficult Times"""

# Enable optimized inference mode for Unsloth models (improves speed and efficiency)
FastLanguageModel.for_inference(model)  # Unsloth has 2x faster inference!

# Format the question using the structured prompt (`prompt_style`) and tokenize it
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")  # Convert input to PyTorch tensor & move to GPU

# Generate a response using the model
outputs = model.generate(
    input_ids=inputs.input_ids, # Tokenized input question
    attention_mask=inputs.attention_mask, # Attention mask to handle padding
    max_new_tokens=1200, # Limit response length to 1200 tokens (to prevent excessive output)
    use_cache=True, # Enable caching for faster inference
)

# Decode the generated output tokens into human-readable text
response = tokenizer.batch_decode(outputs)

# Extract and print only the relevant response part (after "### Response:")
print(response[0].split("### Response:")[1])


<think>
Okay, so I need to come up with a meaningful and relevant quote about gratitude in difficult times. The topic is "Gratitude in Difficult Times," so the quote should reflect how people can find strength, hope, or positive perspective despite challenging circumstances.

First, I should think about well-known personalities who have given motivational quotes. Maybe someone like Albert Einstein, Nelson Mandela, or even more recent figures like Brene Brown or Eckhart Tolle. I recall that Brene Brown has some quotes about gratitude. Alternatively, perhaps someone like Kahlil Gibran or Rumi also touched on gratitude.

I need to capture the essence of not letting difficult times overshadow the positives. Maybe something that emphasizes the power of gratitude in transforming or influencing one's mindset during tough times.

Let me think about some key points: in difficult times, people often lose hope, feel overwhelmed, or become negative. Gratitude can counteract that by shifting focus

**Fine-tuning step by step**



In [ ]:
# Updated training prompt style to add </think> tag
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a motivational quote generator and speaker trained on quotes from well-known personalities.
Given a topic, respond with an inspirational or thoughtful quote that fits well.

### Question:
{}

### Response:
<think>
{}
</think>
{}"""


In [ ]:
# Download the dataset using Hugging Face — function imported using from datasets import load_dataset
dataset = load_dataset("Abirate/english_quotes","default", split = "train[0:1500]",trust_remote_code=True) # Keep only first 1500 rows
dataset

README.md:   0%|          | 0.00/5.55k [00:00<?, ?B/s]

quotes.jsonl:   0%|          | 0.00/647k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2508 [00:00<?, ? examples/s]

Dataset({
    features: ['quote', 'author', 'tags'],
    num_rows: 1500
})

In [ ]:
# Show an entry from the dataset
dataset[1]

{'quote': "“I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best.”",
 'author': 'Marilyn Monroe',
 'tags': ['best',
  'life',
  'love',
  'mistakes',
  'out-of-control',
  'truth',
  'worst']}

In [ ]:
# We need to format the dataset to fit our prompt training style
EOS_TOKEN = "<|endoftext|>"  # Define EOS_TOKEN which the model when to stop generating text during training
EOS_TOKEN

'<|endoftext|>'

In [ ]:
# Define formatting prompt function
def formatting_prompts_func(examples):
    tags_list = examples["tags"]       # Using tags as input "Topic"
    quotes = examples["quote"]         # This will be used as the final response
    authors = examples["author"]       # Include attribution in the final output

    texts = []

    for tags, quote, author in zip(tags_list, quotes, authors):
        topic = ", ".join(tags)  # Combine tags into a single topic string
        cot = f"The user is looking for inspiration around: {topic}."  # Auto-generated CoT based on tags
        output = f"{quote} – {author}"  # Final output with author

        text = train_prompt_style.format(topic, cot, output) + EOS_TOKEN
        texts.append(text)

    return {
        "text": texts
    }

In [ ]:
# Update dataset formatting
dataset_finetune = dataset.map(formatting_prompts_func, batched = True)
dataset_finetune["text"][0]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

'Below is an instruction that describes a task, paired with an input that provides further context. \nWrite a response that appropriately completes the request. \nBefore answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.\n\n### Instruction:\nYou are a motivational quote generator and speaker trained on quotes from well-known personalities. \nGiven a topic, respond with an inspirational or thoughtful quote that fits well. \n\n### Question:\nbe-yourself, gilbert-perreira, honesty, inspirational, misattributed-oscar-wilde, quote-investigator\n\n### Response:\n<think>\nThe user is looking for inspiration around: be-yourself, gilbert-perreira, honesty, inspirational, misattributed-oscar-wilde, quote-investigator.\n</think>\n“Be yourself; everyone else is already taken.” – Oscar Wilde<|endoftext|>'

In [ ]:
# Apply LoRA (Low-Rank Adaptation) fine-tuning to the model
model_lora = FastLanguageModel.get_peft_model(
    model,
    r=16,  # LoRA rank: Determines the size of the trainable adapters (higher = more parameters, lower = more efficiency)
    target_modules=[  # List of transformer layers where LoRA adapters will be applied
        "q_proj",   # Query projection in the self-attention mechanism
        "k_proj",   # Key projection in the self-attention mechanism
        "v_proj",   # Value projection in the self-attention mechanism
        "o_proj",   # Output projection from the attention layer
        "gate_proj",  # Used in feed-forward layers (MLP)
        "up_proj",    # Part of the transformer’s feed-forward network (FFN)
        "down_proj",  # Another part of the transformer’s FFN
    ],
    lora_alpha=16,  # Scaling factor for LoRA updates (higher values allow more influence from LoRA layers)
    lora_dropout=0,  # Dropout rate for LoRA layers (0 means no dropout, full retention of information)
    bias="none",  # Specifies whether LoRA layers should learn bias terms (setting to "none" saves memory)
    use_gradient_checkpointing="unsloth",  # Saves memory by recomputing activations instead of storing them (recommended for long-context fine-tuning)
    random_state=3407,  # Sets a seed for reproducibility, ensuring the same fine-tuning behavior across runs
    use_rslora=False,  # Whether to use Rank-Stabilized LoRA (disabled here, meaning fixed-rank LoRA is used)
    loftq_config=None,  # Low-bit Fine-Tuning Quantization (LoFTQ) is disabled in this configuration
)

Unsloth 2025.3.19 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
# Initialize the fine-tuning trainer — Imported using from trl import SFTTrainer
trainer = SFTTrainer(
    model=model_lora,  # The model to be fine-tuned
    tokenizer=tokenizer,  # Tokenizer to process text inputs
    train_dataset=dataset_finetune,  # Dataset used for training
    dataset_text_field="text",  # Specifies which field in the dataset contains training text
    max_seq_length=max_seq_length,  # Defines the maximum sequence length for inputs
    dataset_num_proc=2,  # Uses 2 CPU threads to speed up data preprocessing

    # Define training arguments
    args=TrainingArguments(
        per_device_train_batch_size=2,  # Number of examples processed per device (GPU) at a time
        gradient_accumulation_steps=4,  # Accumulate gradients over 4 steps before updating weights
        num_train_epochs=1, # Full fine-tuning run
        warmup_steps=5,  # Gradually increases learning rate for the first 5 steps
        max_steps=60,  # Limits training to 60 steps (useful for debugging; increase for full fine-tuning)
        learning_rate=2e-4,  # Learning rate for weight updates (tuned for LoRA fine-tuning)
        fp16=not is_bfloat16_supported(),  # Use FP16 (if BF16 is not supported) to speed up training
        bf16=is_bfloat16_supported(),  # Use BF16 if supported (better numerical stability on newer GPUs)
        logging_steps=10,  # Logs training progress every 10 steps
        optim="adamw_8bit",  # Uses memory-efficient AdamW optimizer in 8-bit mode
        weight_decay=0.01,  # Regularization to prevent overfitting
        lr_scheduler_type="linear",  # Uses a linear learning rate schedule
        seed=3407,  # Sets a fixed seed for reproducibility
        output_dir="outputs",  # Directory where fine-tuned model checkpoints will be saved
    ),
)


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/1500 [00:00<?, ? examples/s]

In [ ]:
# Start the fine-tuning process
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,500 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,2.523600
20,0.736300
30,0.622600
40,0.511700
50,0.616600
60,0.568000


In [ ]:
# Save the fine-tuned model
wandb.finish()

train/epoch,▁▂▄▅▇██
train/global_step,▁▂▄▅▇██
train/grad_norm,█▂▁▁▃▂
train/learning_rate,█▇▅▄▂▁
train/loss,█▂▁▁▁▁
total_flos,4121585957289984.0
train/epoch,0.32
train/global_step,60
train/grad_norm,0.60677
train/learning_rate,0.0
train/loss,0.568


In [ ]:
question = """Gratitude in Difficult Times"""

# Load the inference model using FastLanguageModel (Unsloth optimizes for speed)
FastLanguageModel.for_inference(model_lora)  # Unsloth has 2x faster inference!

# Tokenize the input question with a specific prompt format and move it to the GPU
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

# Generate a response using LoRA fine-tuned model with specific parameters
outputs = model_lora.generate(
    input_ids=inputs.input_ids,          # Tokenized input IDs
    attention_mask=inputs.attention_mask, # Attention mask for padding handling
    max_new_tokens=1200,                  # Maximum length for generated response
    use_cache=True,                        # Enable cache for efficient generation
)

# Decode the generated response from tokenized format to readable text
response = tokenizer.batch_decode(outputs)

# Extract and print only the model's response part after "### Response:"
print(response[0].split("### Response:")[1])


<think>
The user is looking for inspiration around: Gratitude in Difficult Times.
</think>
“Gratitude is the most essential and powerful human emotion.” – Robert Emmons,<|endoftext|><｜end▁of▁sentence｜>


In [ ]:
question = """I am depressed give me motivation to carry on and it will be better"""

# Tokenize the input question with a specific prompt format and move it to the GPU
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

# Generate a response using LoRA fine-tuned model with specific parameters
outputs = model_lora.generate(
    input_ids=inputs.input_ids,          # Tokenized input IDs
    attention_mask=inputs.attention_mask, # Attention mask for padding handling
    max_new_tokens=1200,                  # Maximum length for generated response
    use_cache=True,                        # Enable cache for efficient generation
)

# Decode the generated response from tokenized format to readable text
response = tokenizer.batch_decode(outputs)

# Extract and print only the model's response part after "### Response:"
print(response[0].split("### Response:")[1])


<think>
The user is looking for inspiration around: I am depressed give me motivation to carry on and it will be better.
</think>
“Depression is a liar. It will lie to you and tell you that you can't live without it. But you can live without it. You have to be strong and be on the lookout for the moments when it tries to crawl back in. You have to say, 'I don't need this.' You have to kick it out. You have to tell yourself, 'I'm strong, I can do this.' And if you need to, you can look for someone who can help you kick it out. But you can live without it.” – Sarah Dessen,<|endoftext|><｜end▁of▁sentence｜>


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

# Load your tokenizer
tok = AutoTokenizer.from_pretrained("tiiuae/falcon-7b", trust_remote_code=True)
tok.pad_token = tok.eos_token  # Falcon-7B doesn't have a pad token by default

# Load your dataset (update with your actual path if needed)
dataset = load_dataset("Abirate/english_quotes","default",trust_remote_code=True)

# Tokenize only the 'quote' field
def tokenize_function(examples):
    result = tok(examples["quote"], padding="max_length", truncation=True, max_length=128)
    result["labels"] = result["input_ids"].copy()  # For causal LM
    return result

# Apply tokenizer and remove unused columns like 'author', 'tags'
dataset_tokenized = dataset.map(tokenize_function, batched=True, remove_columns=dataset["train"].column_names)


Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [ ]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.7 MB/s eta 0:00:00


In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=94c53db6c0010dfb3ccc3b4472b0eb7a52685e5264c0300fd5fec454593012b6
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
import evaluate

rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")

preds = []
labels = []

for i in range(50):
    topic = dataset_finetune[i]["text"].split("### Question:\n")[1].split("\n###")[0]
    true_output = dataset_finetune[i]["text"].split("</think>")[-1].replace("<|endoftext|>", "").strip()

    input_text = tokenizer([prompt_style.format(topic, "")], return_tensors="pt").to("cuda")
    output = model_lora.generate(**input_text, max_new_tokens=200)
    decoded = tokenizer.batch_decode(output)[0].split("### Response:")[1].strip()

    preds.append(decoded)
    labels.append(true_output)

# Compute ROUGE
rouge_result = rouge.compute(predictions=preds, references=labels)
bleu_result = bleu.compute(predictions=preds, references=[[l] for l in labels])

print("ROUGE:", rouge_result)
print("BLEU:", bleu_result)

ROUGE: {'rouge1': np.float64(0.1706549713731876), 'rouge2': np.float64(0.03948861680628071), 'rougeL': np.float64(0.1248501629422713), 'rougeLsum': np.float64(0.13679084604790107)}
BLEU: {'bleu': 0.03818548001849232, 'precisions': [0.14281991138910607, 0.04805914972273567, 0.025956649719025955, 0.011933821535123406], 'brevity_penalty': 1.0, 'length_ratio': 2.001564945226917, 'translation_length': 3837, 'reference_length': 1917}


In [ ]:
!pip install -q streamlit pyngrok evaluate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.6 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import streamlit as st
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM

# Example: Replace with your actual model and tokenizer loading
model_lora = AutoModelForCausalLM.from_pretrained("/content/wandb").to("cuda")
tokenizer = AutoTokenizer.from_pretrained("/content/wandb")

prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a motivational quote generator and speaker trained on quotes from well-known personalities.
Given a topic, respond with an inspirational or thoughtful quote that fits well.

### Question:
{}

### Response:
<think>
{}
</think>
{}"""



st.title("💬 Motivational Quote Generator")

question = st.text_area("Enter your question or motivational prompt:")

if st.button("Generate Response"):
    if not question.strip():
        st.warning("Please enter a valid question.")
    else:
        with st.spinner("Generating response..."):
            # Format and tokenize input
            inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

            # Generate response
            outputs = model_lora.generate(
                input_ids=inputs.input_ids,
                attention_mask=inputs.attention_mask,
                max_new_tokens=1200,
                use_cache=True,
            )

            # Decode and extract response
            response = tokenizer.batch_decode(outputs)[0]
            final_response = response.split("### Response:")[1].strip()

            st.success("Here's your motivational response:")
            st.write(final_response)


Overwriting app.py


In [ ]:
!ngrok config add-authtoken "2w623dvox2m59xYSi3CygU2JZuG_6ndXYpdhfVvyevW6K3ULy"

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok
import os

# Run the app
os.system("streamlit run app.py &")
public_url = ngrok.connect(addr="8501")
print("Streamlit app running at:", public_url)


Streamlit app running at: NgrokTunnel: "https://48a2-34-125-114-174.ngrok-free.app" -> "http://localhost:8501"
